In [ ]:
from bs4 import BeautifulSoup
import lxml
import requests
from selenium.webdriver.common.keys import Keys
from webdriver_manager.firefox import GeckoDriverManager
from seleniumwire import webdriver
from tqdm import tqdm
import os
from Function import *

# Download Links
In this notebook you can find the code we used to download the url’s linked to the first 300 pages of best book ever collection from Goodreads.com.

In [ ]:
url = 'https://www.goodreads.com/list/show/1.Best_Books_Ever?page='
url_1 = 'https://www.goodreads.com'

In [ ]:
f = open('links_1.txt', 'a')
count = 100

for i in tqdm(range(1, 301)):
    try:
        cnt = requests.get(url + str(i))
        soup = BeautifulSoup(cnt.content, features = 'lxml')

        #select all the link in a page 
        links = soup.find_all('a', class_="bookTitle")

        #write the link in the file 
        for link in links:
            f.write(url_1 + link.get('href')+'\n')

        # open new file every 100 pages
        if i == count:
            f.close()
            count += 100
            f = open('links_' + str(count)[0] + '.txt', 'a')
            
    except Exception as e:

    #saving information about errors
    log_error = open('error_' +  str(e) + '.txt', 'a')
    log_error.writelines('page_number:  ' + str(i))
    log_error.close()

f.close()

# Download HTML pages

In [ ]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
driver.scopes = '.+goodreads.com/book/show+'

In [ ]:
f = open('links_1.txt', 'r')
Links = f.readlines()
f.close()

In [ ]:
num_page = 1
num_article = 1

for i, link in enumerate(tqdm(Links)):
    try:
        driver.get(link)
        resp = driver.last_request.response.status_code
        #check page response
        if resp == 200:
            #file name
            filename = "HTML_pages_1/page_"+'{0:03}'.format(num_page)+"/article_"+'{0:05}'.format(i+num_article)+".html"
            # make directory
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            #open file
            f_html = open(filename, "a", encoding="utf-8")
            soup = BeautifulSoup(driver.page_source, features="lxml")
            f_html.write(soup.prettify())
            f_html.close()
        else:
            #saving information about bad response
            error_name = "Erros/bad_response_"+str(resp)+".txt"
            os.makedirs(os.path.dirname(error_name), exist_ok=True)
            f_error = open(error_name, 'a')
            f_error.write(link+"\n")
            f_error.close()
        if (i+num_article)%100 == 0:
            num_page += 1
    except Exception as e:

        #saving information about errors
        os.makedirs(os.path.dirname('Errors_parsing'), exist_ok=True)
        log_error = open('Errors_parsing/error_' +  str(e) + '.txt', 'a')
        log_error.writelines(link+"\n")
        log_error.close()

# Parsing

In [ ]:
file_tsv = "Books.tsv"
f = open('links_1.txt', 'r', encoding= "utf-8")
Links = f.readlines()
f.close()

out_file = open(file_tsv, 'a', encoding= "utf-8", newline='') 
tsv_writer = csv.writer(out_file, delimiter='\t')

# Name of columns in tsv file
tsv_writer.writerow(['bookId', 'bookTitle', 'bookSeries', 'bookAuthors', 'ratingValue', 'ratingCount', 'reviewCount', 
                     'Plot', 'NumberofPages', 'PublishingDate', 'Characters', 'Settings', 'url'])

for drk in tqdm(os.listdir("HTML_pages")):
    for page in os.listdir("HTML_pages/"+drk):
        try:

            file = open('HTML_pages/'+drk+'/'+page, encoding= "utf-8")
            soup = BeautifulSoup(file, features = 'lxml')
            file.close()

            tsv_writer.writerow([page[8:-5]]+scrap_book(soup)+[Links[int(page[8:-5])-1]])

        except Exception as e:
            
            #saving information about errors
            os.makedirs('Errors', exist_ok=True)
            log_error = open('Errors/error_' +  str(e).replace(' ', '_') + '.txt', 'a', encoding= "utf-8")
            log_error.writelines(drk+'/'+page+"\n")
            log_error.close()

out_file.close()